In [1]:
import math
import pandas as pd
import numpy as np
import statistics

In [302]:
principal = 500000
rate = 0.05/12
n = 12
data = pd.read_csv("MORTGAGE30US.csv")
vol =statistics.stdev(data['MORTGAGE30US'])*math.sqrt(12)/100
vol

0.1114034576714194

In [303]:
data

,observation_date,MORTGAGE30US
0,1971-04-02,7.33
1,1971-04-09,7.31
2,1971-04-16,7.31
3,1971-04-23,7.31
4,1971-04-30,7.29
...,...,...
2812,2025-02-20,6.85
2813,2025-02-27,6.76
2814,2025-03-06,6.63
2815,2025-03-13,6.65


In [304]:
A = principal * rate * (1+rate)**n/((1+rate)**n-1)
A

42803.74089423373

In [595]:
#for T_10, 1 = stepsTaken, 0 = place
#for T_(n-1)1, n-1 = stepsTaken, 1 = place
#find root.data such that root.data = P

count = 0
class Tree:
    def __init__(self):
        self.left = None
        self.right = None
        self.data = None
        self.parent = None
def makeTree(steps,A,P,vol):
    global count
    rates = np.ones(steps)*rate-(np.array(range(1,steps+1))*0.00005)
    root = Tree()
    root.level = 0
    root.place = 0
    while root.data == None or abs(root.data - P) >= 0.001:
        #recursively make non Callable Tree
        attemptTree(steps,0,0,root,A,rates,vol)
        diff = abs(root.data - P)
        if diff >= 0.001:
            rates = guessRates(steps,rates,P,root.data,root)
        count = 0
    return (root,rates)
            
def attemptTree(steps, stepsTaken, place, parent, A, rates, vol):    
    nextStep = stepsTaken+1
    global count
    count+=1
    
    if stepsTaken == steps-1:
        placeRate = (1+rates[stepsTaken]*math.exp(place*2*vol*math.sqrt(1/12)))      
        parent.data = A/placeRate
    else:
        placeRate = (1+rates[nextStep]*math.exp(place*2*vol*math.sqrt(1/12)))
        parent.left = Tree()
        parent.right = Tree()
        lower = parent.left
        lower.parent = parent
        upper = parent.right
        upper.parent = parent
        attemptTree(steps, nextStep, place, lower, A, rates, vol)
        attemptTree(steps, nextStep, place+1, upper, A, rates, vol)
        average = (lower.data+upper.data)/2
        parent.data = 1/placeRate*(average+A)

def attemptTree2(steps, stepsTaken, place, parent, A, rates, vol):
    nextStep = stepsTaken+1
    global count
    count+=1
    
    if stepsTaken == steps-1:
        placeRate = (1+rates[stepsTaken]*math.exp(place*2*vol*math.sqrt(1/12)))      
        parent.data = A/placeRate
        print(f'Leaf T_{stepsTaken},{place}:{parent.data}')
    else:
        placeRate = (1+rates[nextStep]*math.exp(place*2*vol*math.sqrt(1/12)))
        parent.left = Tree()
        lower = parent.left
        lower.parent = parent
        if lower.data == None:
            attemptTree2(steps,nextStep,place,lower,A,rates,vol)
        parent.right = Tree()
        upper = parent.right
        upper.parent = parent
        upper.left = lower.right
        if upper.data == None:
            if upper.left == None or nextStep >= steps-1:
                attemptTree2(steps, nextStep, place+1, upper, A, rates, vol)
            else:
                attemptTree2(steps, nextStep+1, place+2, upper, A, rates, vol)
        average = (lower.data+upper.data)/2
        parent.data = 1/placeRate*(average+A)
        print(f'T_{stepsTaken},{place}:{parent.data}')
    
def guessRates(steps,prevRates,P,base,tree):
    #ri = ri + (T00-P)/(Ti0)*(convergenceVar)
    target = tree
    for step in range(1,steps):
        target = target.left
        diff = base - P
        change = diff/base*prevRates[step]
        new = prevRates[step] + change
        prevRates[step] = new
    return prevRates

In [593]:
count = 0
root,rates = makeTree(n,A,principal,vol)

4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095


In [594]:
print(root.data)
print(rates)

499999.9990074775
[0.00411667 0.00386791 0.00382035 0.0037728  0.00372524 0.00367768
 0.00363013 0.00358257 0.00353502 0.00348746 0.0034399  0.00339235]


In [330]:
parent = root.left
for i in range(n-1):
    print(parent.data,'\t',parent.parent.right.data)
    parent = parent.left

419503.281498247 	 467262.42828053184
381907.87022693676 	 350512.1726055082
345103.8299351565 	 314125.8897857004
309351.9360105477 	 277412.6718573331
273271.3847547001 	 242607.25263885278
238991.0802255917 	 205137.666197126
202155.31327947232 	 173322.81922162801
170725.3801036605 	 131273.58795757184
129454.85225353648 	 108212.23028922842
106505.6110324703 	 49845.593780359006
49339.943657096555 	 49845.593780359006


In [572]:
def oldPV(steps,A,P,rate):
    leftover = P
    balances = [P]
    PVs = []
    for step in range(1,steps):
        interest = leftover * rate
        paid = A - interest
        leftover = leftover - paid 
        balances.append(leftover)
        PV = interest/(1+rate)**step
        PVs.append(PV)
    return (PVs,balances)

def refinancePV(steps,start,balance,rates):
    rate = rates[start-1]
    n = steps - start + 1
    A = balance * rate * (1+rate)**n/((1+rate)**n-1)
    total = 0
    leftover = balance
    for step in range(start,steps):
        interest = leftover * rate
        paid = A - interest
        leftover = leftover - paid
        total += interest/(1+rate)
    return total

In [179]:
PVs, balances = oldPV(n,A,principal,rate)
print(balances)
print(A)
for i in range(len(PVs)):
    PVrefi = refinancePV(n,i+2,balances[i+1],rates)
    print(f"Refinance at Month {i+1} refiSavings: ",sum(PVs[i:])-PVrefi)

[500000, 459279.5924390996, 418389.5165133621, 377329.0652712674, 336097.52881566394, 294694.1942914955, 253118.34587347633, 211369.26475371543, 169446.22912928884, 127348.51418976049, 85075.39210465076, 42626.13201085308]
42803.74089423373
Refinance at Month 1 refiSavings:  219855.07829580305
Refinance at Month 2 refiSavings:  105856.42220805903
Refinance at Month 3 refiSavings:  65142.90384337915
Refinance at Month 4 refiSavings:  44101.045049595894
Refinance at Month 5 refiSavings:  31243.96002064192
Refinance at Month 6 refiSavings:  22411.189755294406
Refinance at Month 7 refiSavings:  16110.66213336816
Refinance at Month 8 refiSavings:  11018.249025446836
Refinance at Month 9 refiSavings:  7289.572003152549
Refinance at Month 10 refiSavings:  3497.0057294856497
Refinance at Month 11 refiSavings:  338.63271156280445


In [446]:
print((np.ones(n)/(np.array(range(1,13)))**2))

[1.         0.25       0.11111111 0.0625     0.04       0.02777778
 0.02040816 0.015625   0.01234568 0.01       0.00826446 0.00694444]
